In [3]:
from __future__ import division
font = {'size'   : 20}
import matplotlib
matplotlib.rc('font', **font)
import matplotlib.pylab as plt
import numpy as np

In [4]:
def getRows(dataBase,startInd,endInd):
    import sqlite3
    print(type(dataBase))
    input1 = dataBase+'.rdb'
    conn = sqlite3.connect(input1)
    c = conn.cursor()
    input2 = 'SELECT * FROM '+dataBase+'_ob'
    aa = c.execute(input2)
    rows = []
    count = 0
    for row in aa:
        if (count >= startInd) & (count < endInd):
            rows.append(row)
        count += 1
        if count == endInd:
            break
    return rows

In [5]:
def getStartTime(row):
    import datetime as dt
    start = dt.datetime(int(row[5][0:4]), int(row[5][5:7]), int(row[5][8:10]), int(row[5][11:13]), int(row[5][14:16]), int(float(row[5][17:-1])))
    return start

In [6]:
def getTimes(rows,startTime):
    import matplotlib.pyplot as plt
    import datetime as dt
    timeSec = [0]
    for afterInd in range(1,len(rows)):
        current = dt.datetime(int(rows[afterInd][5][0:4]), int(rows[afterInd][5][5:7]), int(rows[afterInd][5][8:10]), int(rows[afterInd][5][11:13]), int(rows[afterInd][5][14:16]), int(float(rows[afterInd][5][17:-1])))
        currentDiff = current - startTime
        currentDiffSeconds = currentDiff.days*(24*3600) + currentDiff.seconds
        timeSec.append(currentDiffSeconds)
    return timeSec

In [7]:
def extractTickData(rows,timeSec):
    import numpy as np
    buyPrices = []
    sellPrices = []
    buyTimes = []
    sellTimes = []
    buyQuantity = []
    buyLevels = []
    sellQuantity = []
    sellLevels = []
    for ii in range(len(rows)):
        if rows[ii][1] == 'buy':
            buyPrices.append(rows[ii][2])
            buyTimes.append(timeSec[ii])
            buyQuantity.append(rows[ii][3])
            buyLevels.append(rows[ii][4])
        else:
            sellPrices.append(rows[ii][2])
            sellTimes.append(timeSec[ii])
            sellQuantity.append(rows[ii][3])
            sellLevels.append(rows[ii][4])
    buyPrices = np.array(buyPrices)
    sellPrices = np.array(sellPrices)
    buyTimes = np.array(buyTimes)
    sellTimes = np.array(sellTimes)
    buyQuantity = np.array(buyQuantity)
    buyLevels = np.array(buyLevels)
    sellQuantity = np.array(sellQuantity)
    sellLevels = np.array(sellLevels)
    
    return buyPrices, sellPrices, buyTimes, sellTimes, buyQuantity, buyLevels, sellQuantity, sellLevels

In [8]:
def DHMS(seconds):
    import numpy as np
    days = np.floor(seconds/3600)
    hours = np.floor((seconds/3600) - days*24)
    minutes = np.floor(((seconds/3600) - days*24 - hours)*60)
    seconds = np.floor((((seconds/3600) - days*24 - hours)*60 - minutes)*60)
    return np.array([days,hours,minutes,seconds])

In [9]:
def plotTimes(sellTimes1,sellPrices1,buyTimes1,buyPrices1,sellTimes2,sellPrices2,buyTimes2,buyPrices2,label1,label2,start):
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots()
    import datetime as dt
    ax.plot(buyTimes1,buyPrices1,'g-',label=label1+' Buy Prices')
    ax.plot(sellTimes1,sellPrices1,'r-',label=label1+' Sell Prices')
    ax.plot(buyTimes2,buyPrices2,'b-',label=label2+' Buy Prices')
    ax.plot(sellTimes2,sellPrices2,'m-',label=label2+' Sell Prices')
    plt.show()
    labels = []
    for ii in range(0,len(ax.get_xticklabels())):
        if ax.get_xticklabels()[ii].get_text() != '':
            timeInc = int(ax.get_xticklabels()[ii].get_text())
            dhms = DHMS(timeInc)
            step = dt.timedelta(days = dhms[0],hours = dhms[1],minutes = dhms[2],seconds = dhms[3]) 
            currentLabel = start + step
            labels.append(currentLabel.strftime('%Y:%m:%d:%H:%M:%S'))
        else:
            labels.append(ax.get_xticklabels()[ii].get_text())
    fig, ax = plt.subplots()
    ax.plot(buyTimes1,buyPrices1,'g-',label=label1+' Buy Prices')
    ax.plot(sellTimes1,sellPrices1,'r-',label=label1+' Sell Prices')
    ax.plot(buyTimes2,buyPrices2,'b-',label=label2+' Buy Prices')
    ax.plot(sellTimes2,sellPrices2,'m-',label=label2+' Sell Prices')
    ax.set_xticklabels(labels)
    plt.xlabel('Time')
    plt.ylabel('Bitcoin Buy and Sell Prices')
    plt.title(label1+' and '+label2+' Bitcoin Bid/Ask')
    plt.legend()
    plt.show()

In [10]:
def timeFilterData(data,times,cutoff1,cutoff2):
    filteredData = data[(cutoff1 < times) & (times < cutoff2)]
    return filteredData

In [11]:
def trainData(sellTimes1, sellPrices1, buyTimes1, buyPrices1, sellTimes2, sellPrices2, buyTimes2, buyPrices2):
    import numpy as np
    from scipy import interpolate
    from sklearn import linear_model
    
    tck1 = interpolate.splrep(buyTimes1, buyPrices1, s=0)
    tck2 = interpolate.splrep(buyTimes2, buyPrices2, s=0)
    tck3 = interpolate.splrep(sellTimes2, sellPrices2, s=0)
    compare_buy_1 = interpolate.splev(sellTimes1, tck1, der=0)
    compare_buy_2 = interpolate.splev(sellTimes1, tck2, der=0)
    compare_sell_2 = interpolate.splev(sellTimes1, tck3, der=0)
    
    clfT = linear_model.LinearRegression()
    formatX = np.transpose(sellPrices1)
    formatY = np.transpose(compare_buy_2)
    newX = np.array([formatX]).T
    clfT.fit(newX, formatY)
    slopeToUse_1 = clfT.coef_
    interceptToUse_1 = clfT.intercept_

    clfT2 = linear_model.LinearRegression()
    formatX = np.transpose(compare_buy_1)
    formatY = np.transpose(compare_sell_2)
    newX = np.array([formatX]).T
    clfT2.fit(newX, formatY)
    slopeToUse_2 = clfT2.coef_
    interceptToUse_2 = clfT2.intercept_
    
    return slopeToUse_1, interceptToUse_1, slopeToUse_2, interceptToUse_2, compare_buy_1, compare_buy_2, compare_sell_2

In [12]:
def formatTradeData(sellTimes1,sellPrices1,buyTimes1,buyPrices1,sellTimes2,sellPrices2,buyTimes2,buyPrices2,sellTimes1_ref,sellPrices1_ref,buyTimes1_ref,buyPrices1_ref,sellTimes2_ref,sellPrices2_ref,buyTimes2_ref,buyPrices2_ref,backwardHours,slope1,int1,slope2,int2):
    
    import numpy as np
    from scipy import interpolate
    from sklearn import linear_model
    from scipy.interpolate import interp1d
    """
    tck1 = interpolate.splrep(buyTimes1, buyPrices1, s=0)
    tck2 = interpolate.splrep(buyTimes2, buyPrices2, s=0)
    tck3 = interpolate.splrep(sellTimes2, sellPrices2, s=0)
    buy_1_sync = interpolate.splev(sellTimes1, tck1, der=0)
    buy_2_sync = interpolate.splev(sellTimes1, tck2, der=0)
    sell_2_sync = interpolate.splev(sellTimes1, tck3, der=0)
    """
    leftPoint = max([buyTimes1[0],buyTimes2[0],sellTimes2[0],sellTimes1[0]])
    rightPoint = min([buyTimes1[-1],buyTimes2[-1],sellTimes2[-1],sellTimes1[-1]])
    trade_timeline = sellTimes1[(sellTimes1 > leftPoint) & (sellTimes1 < rightPoint)]
    
    f1 = interp1d(buyTimes1, buyPrices1)
    f2 = interp1d(buyTimes2, buyPrices2)
    f3 = interp1d(sellTimes2, sellPrices2)
    f4 = interp1d(sellTimes1, sellPrices1)
    buy_1_sync = f1(trade_timeline)
    buy_2_sync = f2(trade_timeline)
    sell_2_sync = f3(trade_timeline)
    sell_1_sync = f4(trade_timeline)
    
    leftPoint_ref = max([buyTimes1_ref[0],buyTimes2_ref[0],sellTimes2_ref[0],sellTimes1_ref[0]])
    rightPoint_ref = min([buyTimes1_ref[-1],buyTimes2_ref[-1],sellTimes2_ref[-1],sellTimes1_ref[-1]])
    time_ref = sellTimes1_ref[(sellTimes1_ref > leftPoint_ref) & (sellTimes1_ref < rightPoint_ref)]
    
    """       
    tck1_ref = interpolate.splrep(buyTimes1_ref, buyPrices1_ref, s=0)
    tck2_ref = interpolate.splrep(buyTimes2_ref, buyPrices2_ref, s=0)
    tck3_ref = interpolate.splrep(sellTimes2_ref, sellPrices2_ref, s=0)
    buy_1_ref_sync = interpolate.splev(time_ref, tck1_ref, der=0)
    buy_2_ref_sync = interpolate.splev(time_ref, tck2_ref, der=0)
    sell_2_ref_sync = interpolate.splev(time_ref, tck3_ref, der=0)
    """
    g1 = interp1d(buyTimes1_ref, buyPrices1_ref)
    g2 = interp1d(buyTimes2_ref, buyPrices2_ref)
    g3 = interp1d(sellTimes2_ref, sellPrices2_ref)
    g4 = interp1d(sellTimes1_ref, sellPrices1_ref)
    
    buy_1_ref_sync = g1(time_ref)
    buy_2_ref_sync = g2(time_ref)
    sell_2_ref_sync = g3(time_ref)
    sell_1_ref_sync = g4(time_ref)
    
    zScore1 = np.zeros(len(trade_timeline)) 
    zScore2 = np.zeros(len(trade_timeline)) 
    
    import time
    startTime = time.time()
    time0 = trade_timeline[0]
    count = 0
    calcFlag = True
    for ii in range(0,len(zScore1)):
        if ii == 10**4:
            endTime = time.time()
            # print(endTime-startTime)
            # break
        currentTime = trade_timeline[ii]
        if (calcFlag == True):
            count = count + 1
            calcFlag = False
            sellPrices1_past = sell_1_ref_sync[time_ref < currentTime]
            buyPrices2_past = buy_2_ref_sync[time_ref < currentTime]
            time_past = time_ref[time_ref < currentTime]
           
            time_filt = time_past - time_past[-1]

            sellPrices1_past2 = sellPrices1_past[time_filt > -3600*backwardHours]
            buyPrices2_past2 = buyPrices2_past[time_filt > -3600*backwardHours]
            
            standardDev_1 = np.std((slope1*sellPrices1_past2 + int1) - buyPrices2_past2)
            meanHist_1 = np.mean((slope1*sellPrices1_past2 + int1) - buyPrices2_past2)

            buyPrices1_past = buy_1_ref_sync[time_ref < currentTime]
            sellPrices2_past = sell_2_ref_sync[time_ref < currentTime]

            buyPrices1_past2 = buyPrices1_past[time_filt > -3600*backwardHours]
            sellPrices2_past2 = sellPrices2_past[time_filt > -3600*backwardHours]

            standardDev_2 = np.std((slope2*buyPrices1_past2 + int2) - sellPrices2_past2)
            meanHist_2 = np.mean((slope2*buyPrices1_past2 + int2) - sellPrices2_past2)
        else:
            if (currentTime - time0 > 1*60*count):
                calcFlag = True
       
        zScore1[ii] = (((slope1*sell_1_sync[ii] + int1) - buy_2_sync[ii]) - meanHist_1)/standardDev_1
        zScore2[ii] = (((slope2*buy_1_sync[ii] + int2) - sell_2_sync[ii]) - meanHist_2)/standardDev_2
        """
        if (abs(zScore1[ii]) > 10) | (abs(zScore2[ii]) > 10):
            print(len(buyPrices1_past2))
            print(currentTime)
            print(time_past[-10:-1])
            print(currentTime - 3600*backwardHours)
        """
            
    return zScore1, zScore2, trade_timeline, sell_1_sync, buy_1_sync, buy_2_sync, sell_2_sync 

In [135]:
def pairs_trade(timeLine,sellPrices1,buyPrices1,sellPrices2,buyPrices2,zScore_1,zScore_2,trade_cutoff):
    import numpy as np
    bank1 = 5000
    bank2 = 5000
    coins1 = 0
    coins2 = 0

    bankTracker1 = []
    coinTracker1 = []

    bankTracker2 = []
    coinTracker2 = []

    zScoreTracker_1 = []
    zScoreTracker_2 = []
    cashOutTimes = []
    cashOut1 = []
    cashOut2 = []
    cashOutz1 = []
    cashOutz2 = []
    
    truthDir = []
    predDir = []
    
    posTradingDiffs = []
    negTradingDiffs = []
    
    extremeFlag = 0
    betSize = .1
    
    for ii in range(0,len(timeLine)-1):
        
        currentTime = timeLine[ii]
        sellP1 = sellPrices1[ii]
        sellP2 = sellPrices2[ii]
        buyP1 = buyPrices1[ii]
        buyP2 = buyPrices2[ii]
        
        if zScore_1[ii] < -trade_cutoff:
            bank2 = bank2 + betSize*buyP2 #sell
            #bank2 = bank2 - 0.002*betSize*buyP2
            coins2 = coins2 - betSize
            bank1 = bank1 - betSize*sellP1 #buy
            #bank1 = bank1 - 0.002*betSize*sellP1
            coins1 = coins1 + betSize
            predDir.append(1)
            truthDir.append(np.sign((sellPrices1[ii+1]-buyPrices2[ii+1])-(sellP1-buyP2)))
            posTradingDiffs.append((sellPrices1[ii+1]-buyPrices2[ii+1])-(sellP1-buyP2))
            extremeFlag = 1
        if zScore_2[ii] > trade_cutoff:
            bank1 = bank1 + betSize*buyP1 #sell
            #bank1 = bank1 - 0.002*betSize*buyP1
            coins1 = coins1 - betSize
            bank2 = bank2 - betSize*sellP2 #buy
            #bank2 = bank2 - 0.002*betSize*sellP2
            coins2 = coins2 + betSize
            predDir.append(-1)
            truthDir.append(np.sign((buyPrices1[ii+1]-sellPrices2[ii+1])-(buyP1-sellP2)))
            negTradingDiffs.append((buyPrices1[ii+1]-sellPrices2[ii+1])-(buyP1-sellP2))
            extremeFlag = 1
        if (abs(zScore_1[ii]) < .5) & (abs(zScore_2[ii]) < .5):
        
            if extremeFlag == 1:
                cashOutTimes.append(currentTime)
                if coins1 > 0:
                    bank1 = bank1 + coins1*buyP1
                    #bank1 = bank1 - 0.002*coins1*buyP1
                else:
                    bank1 = bank1 + coins1*sellP1
                    #bank1 = bank1 + 0.002*coins1*sellP1
                if coins2 > 0:
                    bank2 = bank2 + coins2*buyP2
                    #bank2 = bank2 - 0.002*coins2*buyP2
                else:
                    bank2 = bank2 + coins2*sellP2 
                    #bank2 = bank2 + 0.002*coins2*sellP2
                coins1 = 0
                coins2 = 0
                extremeFlag = 0
                cashOut1.append(bank1)
                cashOut2.append(bank2)
                cashOutz1.append(zScore_1[ii])
                cashOutz2.append(zScore_2[ii])
                
        bankTracker1.append(bank1)
        coinTracker1.append(coins1)

        bankTracker2.append(bank2)
        coinTracker2.append(coins2)
        
    currentTime = timeLine[ii]
    sellP1 = sellPrices1[-1]
    sellP2 = sellPrices2[-1]
    buyP1 = buyPrices1[-1]
    buyP2 = buyPrices2[-1]
    
    if coins1 > 0:
        bank1 = bank1 + coins1*buyP1
        #bank1 = bank1 - 0.002*coins1*buyP1
    else:
        bank1 = bank1 + coins1*sellP1
        #bank1 = bank1 + 0.002*coins1*sellP1
    if coins2 > 0:
        bank2 = bank2 + coins2*buyP2
        #bank2 = bank2 - 0.002*coins2*buyP2
    else:
        bank2 = bank2 + coins2*sellP2 
        #bank2 = bank2 + 0.002*coins2*sellP2    
    coins1 = 0
    coins2 = 0 
    cashOutTimes.append(currentTime)
    cashOut1.append(bank1)
    cashOut2.append(bank2)
    cashOutz1.append(zScore_1[-1])
    cashOutz2.append(zScore_2[-1])
    bankTracker1.append(bank1)
    coinTracker1.append(coins1)

    bankTracker2.append(bank2)
    coinTracker2.append(coins2)

    bankTracker1 = np.array(bankTracker1)
    coinTracker1 = np.array(coinTracker1)

    bankTracker2 = np.array(bankTracker2)
    coinTracker2 = np.array(coinTracker2)
    
    cashOutTimes = np.array(cashOutTimes)
    cashOut1 = np.array(cashOut1)
    cashOut2 = np.array(cashOut2)
    cashOutz1 = np.array(cashOutz1)
    cashOutz2 = np.array(cashOutz2)
    
    return bankTracker1, bankTracker2, coinTracker1, coinTracker2, posTradingDiffs, negTradingDiffs, truthDir, predDir, cashOutTimes, cashOut1, cashOut2, cashOutz1, cashOutz2

In [14]:
def plotReturn(timeLine,bankTracker1,bankTracker2,startTime,cashOutTimes,cashOutBank,label1,label2):
    import numpy as np
    fig, ax = plt.subplots()
    ax.plot(timeLine,bankTracker1+bankTracker2,'b-',label=u'Total Portfolio Pairs Trading')
    ax.plot(timeLine,10**4*np.ones(len(timeLine)),'r-',label=u'Total Inactive Trading')
    ax.plot(cashOutTimes,cashOutBank,'g*',label=u'Pairs Trading Cash Out')
    plt.show()
    labels = []
    for ii in range(0,len(ax.get_xticklabels())):
        if ax.get_xticklabels()[ii].get_text() != '':
            timeInc = int(ax.get_xticklabels()[ii].get_text())
            dhms = DHMS(timeInc)
            step = dt.timedelta(days = dhms[0],hours = dhms[1],minutes = dhms[2],seconds = dhms[3]) 
            currentLabel = startTime + step
            labels.append(currentLabel.strftime('%m/%d %H:%M'))
        else:
            labels.append(ax.get_xticklabels()[ii].get_text())
    fig, ax = plt.subplots()
    ax.plot(timeLine,bankTracker1+bankTracker2,'b-',label=u'Total Portfolio Pairs Trading')
    ax.plot(timeLine,10**4*np.ones(len(timeLine)),'r-',label=u'Total Inactive Trading')
    ax.plot(cashOutTimes,cashOutBank,'g*',label=u'Pairs Trading Cash Out')
    ax.set_xticklabels(labels)
    plt.xlabel('Time')
    plt.ylabel('Account Value')
    plt.title(label1+' and '+label2+' Pairs Trading')
    plt.legend(loc = 0)
    plt.show()

In [15]:
rows1 = getRows('huobi',20*10**6,27*10**6)

<type 'str'>


In [16]:
len(rows1)

7000000

In [17]:
rows2 = getRows('okcoin',0,4*10**6)

<type 'str'>


In [18]:
len(rows2)

3132800

In [19]:
print(getStartTime(rows1[0]))

2013-12-31 15:25:23


In [20]:
print(getStartTime(rows2[0]))

2014-01-01 21:59:52


In [21]:
print(getStartTime(rows1[-1]))

2014-01-08 11:43:38


In [22]:
print(getStartTime(rows2[-1]))

2014-01-08 00:51:33


In [23]:
rows0 = getRows('huobi',20*10**6,20*10**6 + 10)

<type 'str'>


In [24]:
startTime = getStartTime(rows0[0])
print(startTime)

2013-12-31 15:25:23


In [25]:
times1 = getTimes(rows1,startTime)
times2 = getTimes(rows2,startTime)
buyPrices1, sellPrices1, buyTimes1, sellTimes1, buyQuantity1, buyLevels1, sellQuantity1, sellLevels1 = extractTickData(rows1,times1)
buyPrices2, sellPrices2, buyTimes2, sellTimes2, buyQuantity2, buyLevels2, sellQuantity2, sellLevels2 = extractTickData(rows2,times2)

In [26]:
plotTimes(sellTimes1,sellPrices1,buyTimes1,buyPrices1,sellTimes2,sellPrices2,buyTimes2,buyPrices2,'huobi','okcoin',startTime)

In [27]:
import datetime as dt
startCutoff = dt.datetime(2014,1,2,10,0,0) - startTime
startCutOffSeconds = startCutoff.days*(24*3600) + startCutoff.seconds
trainCutoff = dt.datetime(2014,1,2,23,0,0) - startTime
trainCutoffSeconds = trainCutoff.days*(24*3600) + trainCutoff.seconds
endCutoff = dt.datetime(2014,1,4,2,0,0) - startTime
endCutoffSeconds = endCutoff.days*(24*3600) + endCutoff.seconds

In [207]:
import datetime as dt
startCutoff = dt.datetime(2014,1,6,0,0,0) - startTime
startCutOffSeconds = startCutoff.days*(24*3600) + startCutoff.seconds
trainCutoff = dt.datetime(2014,1,6,10,18,42) - startTime
trainCutoffSeconds = trainCutoff.days*(24*3600) + trainCutoff.seconds
endCutoff = dt.datetime(2014,1,7,9,50,14) - startTime
endCutoffSeconds = endCutoff.days*(24*3600) + endCutoff.seconds

In [195]:
import datetime as dt
startCutoff = dt.datetime(2014,1,7,0,0,0) - startTime
startCutOffSeconds = startCutoff.days*(24*3600) + startCutoff.seconds
trainCutoff = dt.datetime(2014,1,7,9,50,16) - startTime
trainCutoffSeconds = trainCutoff.days*(24*3600) + trainCutoff.seconds
endCutoff = dt.datetime(2014,1,8,0,51,33) - startTime
endCutoffSeconds = endCutoff.days*(24*3600) + endCutoff.seconds

In [208]:
buyPrices1_train = timeFilterData(buyPrices1,buyTimes1,startCutOffSeconds,trainCutoffSeconds)
sellPrices1_train = timeFilterData(sellPrices1,sellTimes1,startCutOffSeconds,trainCutoffSeconds)
buyTimes1_train = timeFilterData(buyTimes1,buyTimes1,startCutOffSeconds,trainCutoffSeconds)
sellTimes1_train = timeFilterData(sellTimes1,sellTimes1,startCutOffSeconds,trainCutoffSeconds)
buyQuantity1_train = timeFilterData(buyQuantity1,buyTimes1,startCutOffSeconds,trainCutoffSeconds)
buyLevels1_train = timeFilterData(buyLevels1,buyTimes1,startCutOffSeconds,trainCutoffSeconds)
sellQuantity1_train = timeFilterData(sellQuantity1,sellTimes1,startCutOffSeconds,trainCutoffSeconds)
sellLevels1_train = timeFilterData(sellLevels1,sellTimes1,startCutOffSeconds,trainCutoffSeconds)

buyPrices2_train = timeFilterData(buyPrices2,buyTimes2,startCutOffSeconds,trainCutoffSeconds)
sellPrices2_train = timeFilterData(sellPrices2,sellTimes2,startCutOffSeconds,trainCutoffSeconds)
buyTimes2_train = timeFilterData(buyTimes2,buyTimes2,startCutOffSeconds,trainCutoffSeconds)
sellTimes2_train = timeFilterData(sellTimes2,sellTimes2,startCutOffSeconds,trainCutoffSeconds)
buyQuantity2_train = timeFilterData(buyQuantity2,buyTimes2,startCutOffSeconds,trainCutoffSeconds)
buyLevels2_train = timeFilterData(buyLevels2,buyTimes2,startCutOffSeconds,trainCutoffSeconds)
sellQuantity2_train = timeFilterData(sellQuantity2,sellTimes2,startCutOffSeconds,trainCutoffSeconds)
sellLevels2_train = timeFilterData(sellLevels2,sellTimes2,startCutOffSeconds,trainCutoffSeconds)

buyPrices1_trade = timeFilterData(buyPrices1,buyTimes1,trainCutoffSeconds,endCutoffSeconds)
sellPrices1_trade = timeFilterData(sellPrices1,sellTimes1,trainCutoffSeconds,endCutoffSeconds)
buyTimes1_trade = timeFilterData(buyTimes1,buyTimes1,trainCutoffSeconds,endCutoffSeconds)
sellTimes1_trade = timeFilterData(sellTimes1,sellTimes1,trainCutoffSeconds,endCutoffSeconds)
buyQuantity1_trade = timeFilterData(buyQuantity1,buyTimes1,trainCutoffSeconds,endCutoffSeconds)
buyLevels1_trade = timeFilterData(buyLevels1,buyTimes1,trainCutoffSeconds,endCutoffSeconds)
sellQuantity1_trade = timeFilterData(sellQuantity1,sellTimes1,trainCutoffSeconds,endCutoffSeconds)
sellLevels1_trade = timeFilterData(sellLevels1,sellTimes1,trainCutoffSeconds,endCutoffSeconds)

buyPrices2_trade = timeFilterData(buyPrices2,buyTimes2,trainCutoffSeconds,endCutoffSeconds)
sellPrices2_trade = timeFilterData(sellPrices2,sellTimes2,trainCutoffSeconds,endCutoffSeconds)
buyTimes2_trade = timeFilterData(buyTimes2,buyTimes2,trainCutoffSeconds,endCutoffSeconds)
sellTimes2_trade = timeFilterData(sellTimes2,sellTimes2,trainCutoffSeconds,endCutoffSeconds)
buyQuantity2_trade = timeFilterData(buyQuantity2,buyTimes2,trainCutoffSeconds,endCutoffSeconds)
buyLevels2_trade = timeFilterData(buyLevels2,buyTimes2,trainCutoffSeconds,endCutoffSeconds)
sellQuantity2_trade = timeFilterData(sellQuantity2,sellTimes2,trainCutoffSeconds,endCutoffSeconds)
sellLevels2_trade = timeFilterData(sellLevels2,sellTimes2,trainCutoffSeconds,endCutoffSeconds)

In [209]:
sellTimes1_train =  sellTimes1_train[sellLevels1_train == 1]
sellPrices1_train = sellPrices1_train[sellLevels1_train == 1]
buyTimes1_train = buyTimes1_train[buyLevels1_train == 1]
buyPrices1_train = buyPrices1_train[buyLevels1_train == 1]
sellTimes2_train = sellTimes2_train[sellLevels2_train == 1]
sellPrices2_train = sellPrices2_train[sellLevels2_train == 1]
buyTimes2_train = buyTimes2_train[buyLevels2_train == 1]
buyPrices2_train = buyPrices2_train[buyLevels2_train == 1]
slopeToUse_1, interceptToUse_1, slopeToUse_2, interceptToUse_2,buyPrices1_train_syn, buyPrices2_train_sync, sellPrices2_train_sync = trainData(sellTimes1_train, sellPrices1_train, buyTimes1_train, buyPrices1_train, sellTimes2_train, sellPrices2_train, buyTimes2_train, buyPrices2_train)

In [174]:
slopeToUse_1

array([ 0.96707862])

In [175]:
interceptToUse_1

155.03960923266186

In [176]:
slopeToUse_2

array([ 0.97132063])

In [177]:
interceptToUse_2

132.97560809887091

In [34]:
import matplotlib.pyplot as plt
plt.plot(sellPrices1_train,buyPrices2_train_sync,'b.')
plt.plot(sellPrices1_train,slopeToUse_1*sellPrices1_train + interceptToUse_1,'r')
plt.xlabel('Huobi sell prices')
plt.ylabel('Okcoin sell prices')
plt.title('Best Fit Parameters')
plt.show()

In [210]:
buyPrices1_trade = buyPrices1_trade[buyLevels1_trade == 1]
sellPrices1_trade = sellPrices1_trade[sellLevels1_trade == 1]
buyTimes1_trade = buyTimes1_trade[buyLevels1_trade == 1]
sellTimes1_trade = sellTimes1_trade[sellLevels1_trade == 1]

buyPrices1_ref = buyPrices1[buyLevels1 == 1]
sellPrices1_ref = sellPrices1[sellLevels1 == 1]
buyTimes1_ref = buyTimes1[buyLevels1 == 1]
sellTimes1_ref = sellTimes1[sellLevels1 == 1]

buyPrices2_trade = buyPrices2_trade[buyLevels2_trade == 1]
sellPrices2_trade = sellPrices2_trade[sellLevels2_trade == 1]
buyTimes2_trade = buyTimes2_trade[buyLevels2_trade == 1]
sellTimes2_trade = sellTimes2_trade[sellLevels2_trade == 1]

buyPrices2_ref = buyPrices2[buyLevels2 == 1]
sellPrices2_ref = sellPrices2[sellLevels2 == 1]
buyTimes2_ref = buyTimes2[buyLevels2 == 1]
sellTimes2_ref = sellTimes2[sellLevels2 == 1]

In [211]:
backwardHours = 6
zScore1,zScore2,timeLine,sellPrices1_trade_sync,buyPrices1_trade_sync,sellPrices2_trade_sync,buyPrices2_trade_sync = formatTradeData(sellTimes1_trade,sellPrices1_trade,buyTimes1_trade,buyPrices1_trade,sellTimes2_trade,sellPrices2_trade,buyTimes2_trade,buyPrices2_trade,sellTimes1_ref,sellPrices1_ref,buyTimes1_ref,buyPrices1_ref,sellTimes2_ref,sellPrices2_ref,buyTimes2_ref,buyPrices2_ref,backwardHours,slopeToUse_1,interceptToUse_1,slopeToUse_2,interceptToUse_2)

In [212]:
trade_cutoff = 0
bankTracker1, bankTracker2, coinTracker1, coinTracker2, posTradingDiffs, negTradingDiffs, truthDir, predDir, cashOutTimes,cashOut1, cashOut2, cashOutz1, cashOutz2 = pairs_trade(timeLine,sellPrices1_trade_sync,buyPrices1_trade_sync,sellPrices2_trade_sync,buyPrices2_trade_sync,zScore1,zScore2,trade_cutoff)

In [213]:
print "correct: ",(np.sum(np.array(posTradingDiffs) > 0)+np.sum(np.array(negTradingDiffs) < 0))/(len(posTradingDiffs)+len(negTradingDiffs))
print "wrong: ",(np.sum(np.array(posTradingDiffs) < 0)+np.sum(np.array(negTradingDiffs) > 0))/(len(posTradingDiffs)+len(negTradingDiffs))
print "zeros: ",(np.sum(np.array(posTradingDiffs) == 0)+np.sum(np.array(negTradingDiffs) == 0))/(len(posTradingDiffs)+len(negTradingDiffs))

correct:  0.286784162964
wrong:  0.308003395077
zeros:  0.405212441959


In [216]:
sum(np.array(predDir) == np.array(truthDir))/len(truthDir)

0.28678416296370263

In [214]:
profit = bankTracker1[-1] + bankTracker2[-1] - 5000 - 5000
print "Profit: $", profit

Profit: $ 56699.2791111


In [116]:
100*((10**4 + profit) / 10**4) - 100

0.35739907090092515

In [130]:
fig, ax = plt.subplots()
xaxis = timeLine
ax.plot(timeLine,bankTracker1+bankTracker2,'r',label='pairs trade')
ax.plot(timeLine,10000*np.ones(len(bankTracker1)),'b',label='do nothing')
plt.show()
labels = []
for ii in range(0,len(ax.get_xticklabels())):
    if ax.get_xticklabels()[ii].get_text() != '':
        timeInc = int(ax.get_xticklabels()[ii].get_text())
        dhms = DHMS(timeInc)
        step = dt.timedelta(days = dhms[0],hours = dhms[1],minutes = dhms[2],seconds = dhms[3]) 
        currentLabel = startTime + step
        labels.append(currentLabel.strftime('%m/%d %H:%M'))
    else:
        labels.append(ax.get_xticklabels()[ii].get_text())
fig, ax = plt.subplots()
ax.plot(timeLine,bankTracker1+bankTracker2,'r',label='pairs trade')
ax.plot(timeLine,10000*np.ones(len(bankTracker1)),'b',label='do nothing')
plt.legend(loc=(0,.8))
plt.ylabel("price")
plt.xlabel("time")
plt.title("BTC Account Value with Pairs Trading")
ax.set_xticklabels(labels)
plt.show()

In [131]:
plotReturn(timeLine,bankTracker1,bankTracker2,startTime,cashOutTimes,cashOut1+cashOut2,'huobi','okcoin')

In [119]:
fig, ax = plt.subplots()
ax.plot(timeLine,zScore1,'r',label=u'btce')
ax.plot(timeLine,zScore2,'b',label=u'bitstamp')
ax.plot(cashOutTimes,cashOutz1,'yo',label=u'btce cash out')
ax.plot(cashOutTimes,cashOutz2,'wo',label=u'bitstamp cash out')
plt.show()
labels = []
for ii in range(0,len(ax.get_xticklabels())):
    if ax.get_xticklabels()[ii].get_text() != '':
        timeInc = int(ax.get_xticklabels()[ii].get_text())
        dhms = DHMS(timeInc)
        step = dt.timedelta(days = dhms[0],hours = dhms[1],minutes = dhms[2],seconds = dhms[3]) 
        currentLabel = startTime + step
        labels.append(currentLabel.strftime('%Y:%m:%d:%H:%M:%S'))
    else:
        labels.append(ax.get_xticklabels()[ii].get_text())
fig, ax = plt.subplots()
ax.plot(timeLine,zScore1,'r',label=u'btce')
ax.plot(timeLine,zScore2,'b',label=u'bitstamp')
ax.plot(cashOutTimes,cashOutz1,'yo',label=u'btce cash out')
ax.plot(cashOutTimes,cashOutz2,'wo',label=u'bitstamp cash out')   
ax.set_xticklabels(labels)
plt.xlabel('Time')
plt.ylabel('Z Score')
plt.title('Huobi and okcoin Z scores with Cashout times (z_threshold = 2)')
plt.legend(loc = 0)
plt.show()

In [132]:
fig, ax = plt.subplots()
ax.plot(timeLine,coinTracker1,'r',label='huobi')
ax.plot(timeLine,coinTracker2,'b',label='okcoin')
plt.show()
labels = []
for ii in range(0,len(ax.get_xticklabels())):
    if ax.get_xticklabels()[ii].get_text() != '':
        timeInc = int(ax.get_xticklabels()[ii].get_text())
        dhms = DHMS(timeInc)
        step = dt.timedelta(days = dhms[0],hours = dhms[1],minutes = dhms[2],seconds = dhms[3]) 
        currentLabel = startTime + step
        labels.append(currentLabel.strftime('%Y:%m:%d:%H:%M:%S'))
    else:
        labels.append(ax.get_xticklabels()[ii].get_text())
fig, ax = plt.subplots()
ax.plot(timeLine,coinTracker1,'r',label='huobi')
ax.plot(timeLine,coinTracker2,'b',label='okcoin') 
ax.set_xticklabels(labels)
plt.xlabel('Time')
plt.ylabel('Bitcoins')
plt.title('Huobi and Okcoin Bitcoins with Cashout times (z_threshold = 2)')
plt.legend(loc = 0)
plt.show()

In [134]:
fig, ax = plt.subplots()
ax.plot(timeLine,bankTracker1,'r',label='huobi')
ax.plot(timeLine,bankTracker2,'b',label='okcoin')
plt.show()
labels = []
for ii in range(0,len(ax.get_xticklabels())):
    if ax.get_xticklabels()[ii].get_text() != '':
        timeInc = int(ax.get_xticklabels()[ii].get_text())
        dhms = DHMS(timeInc)
        step = dt.timedelta(days = dhms[0],hours = dhms[1],minutes = dhms[2],seconds = dhms[3]) 
        currentLabel = startTime + step
        labels.append(currentLabel.strftime('%Y:%m:%d:%H:%M:%S'))
    else:
        labels.append(ax.get_xticklabels()[ii].get_text())
fig, ax = plt.subplots()
ax.plot(timeLine,bankTracker1,'r',label='huobi')
ax.plot(timeLine,bankTracker2,'b',label='okcoin') 
ax.set_xticklabels(labels)
plt.xlabel('Time')
plt.ylabel('Bitcoins')
plt.title('Huobi and Okcoin Bitcoins with Cashout times (z_threshold = 2)')
plt.legend(loc = 0)
plt.show()

In [99]:
import numpy as np
backwardHours = 6
trade_cutoff_vector = np.linspace(.6,4,40)
tradeReturn = np.zeros(len(trade_cutoff_vector))
zScore1,zScore2,timeLine,sellPrices1_trade_sync,buyPrices1_trade_sync,sellPrices2_trade_sync,buyPrices2_trade_sync = formatTradeData(sellTimes1_trade,sellPrices1_trade,buyTimes1_trade,buyPrices1_trade,sellTimes2_trade,sellPrices2_trade,buyTimes2_trade,buyPrices2_trade,sellTimes1_ref,sellPrices1_ref,buyTimes1_ref,buyPrices1_ref,sellTimes2_ref,sellPrices2_ref,buyTimes2_ref,buyPrices2_ref,backwardHours,slopeToUse_1,interceptToUse_1,slopeToUse_2,interceptToUse_2)
for ii in range(0,len(trade_cutoff_vector)): 
    trade_cutoff = trade_cutoff_vector[ii]
    bankTracker1, naiveWorthTracker1, bankTracker2, naiveWorthTracker2, coinTracker1, coinTracker2, cashOutTimes,cashOut1, cashOut2, cashOutz1, cashOutz2 = pairs_trade(timeLine,sellPrices1_trade_sync,buyPrices1_trade_sync,sellPrices2_trade_sync,buyPrices2_trade_sync,zScore1,zScore2,trade_cutoff)
    tradeReturn[ii] = 100*(((bankTracker1[-1]+bankTracker2[-1])/20000)-1)
plt.plot(trade_cutoff_vector,tradeReturn,'b')
plt.xlabel('Z Score Threshold')
plt.ylabel('Percent Return')
plt.title('Pairs Trading Return')
plt.show()

KeyboardInterrupt: 

In [34]:
import numpy as np
backwardHoursVec = [1/12,1/2,1,2,3,4,5,6,7,8,9,10,11,12]
trade_cutoff_vector = np.linspace(.6,4,20)
tradeReturn = np.zeros((14,len(trade_cutoff_vector)))
for ii in range(0,14):
    backwardHours = backwardHoursVec[ii]
    zScore1,zScore2,timeLine,sellPrices1_trade_sync,buyPrices1_trade_sync,sellPrices2_trade_sync,buyPrices2_trade_sync = formatTradeData(sellTimes1_trade,sellPrices1_trade,buyTimes1_trade,buyPrices1_trade,sellTimes2_trade,sellPrices2_trade,buyTimes2_trade,buyPrices2_trade,sellTimes1_ref,sellPrices1_ref,buyTimes1_ref,buyPrices1_ref,sellTimes2_ref,sellPrices2_ref,buyTimes2_ref,buyPrices2_ref,backwardHours,slopeToUse_1,interceptToUse_1,slopeToUse_2,interceptToUse_2)
    for jj in range(len(trade_cutoff_vector)):
        trade_cutoff = trade_cutoff_vector[jj]
        bankTracker1, naiveWorthTracker1, bankTracker2, naiveWorthTracker2, coinTracker1, coinTracker2, cashOutTimes,cashOut1, cashOut2, cashOutz1, cashOutz2 = pairs_trade(timeLine,sellPrices1_trade_sync,buyPrices1_trade_sync,sellPrices2_trade_sync,buyPrices2_trade_sync,zScore1,zScore2,trade_cutoff)
        tradeReturn[ii,jj] = 100*(((bankTracker1[-1]+bankTracker2[-1])/20000)-1)

In [35]:
fig, ax = plt.subplots()
ax.imshow(tradeReturn)
xlabels = [item.get_text() for item in ax.get_xticklabels()]
ylabels = [item.get_text() for item in ax.get_yticklabels()]
ax.set_xticklabels(['','0.6','1.6','2.6','3.6',''])
ax.set_yticklabels(['','1/12','1','2','4','6','8','10','12',''])
plt.xlabel('Z-score threshold')
plt.ylabel('Backward Window (hours)')
plt.title('huobi and okcoin Pairs Trading Return')
plt.colorbar
plt.show()